# Bloomberg market data API

В этом ноутбуке приведен весь необходимый функционал для скачивания рыночных данных из блумберга и последующей трансформации - adjust on dividends.

# ATTENTION!
Bloomberg API может присылать пустой ответ на запрос о ценах по неясной клиенту причине.

In [1]:
import blpapiwrapper as bw
import pandas as pd
import datetime
import numpy as np
import re
import tqdm
import warnings

import matplotlib.pyplot as plt
%matplotlib inline

### ISIN vs Bloomberg symbol

In this API  you can use both, but we recommend to ISIN ID instead of bloomberg symbol. For this reason we need following function.

In [2]:
def symbol_handler(symbol):
    """
    Returns bwapiwrapper feedable ID for a stock
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'    
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    if not re.search('^[A-Z]{2}(?:|[A-Z0-9]{10})$', symbol):
        warnings.warn("Using ISIN ID is recommended. You use {} as a symbol".format(symbol))   
        sym = symbol
    else:
        sym = symbol + ' EQUITY'
    return sym

In [3]:
def company_name(symbol):
    """
    Returns company name for a stoock
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'    
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    try:
        sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[sym],
                     fields=["NAME"])
        blpts.get()
        blpts.closeSession()
        return str(blpts.output.ix[0, 0])
    except:
        return symbol

In [4]:
def ticker(symbol):
    """
    Returns company name for a stoock
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'    
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    try:
        sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[sym],
                     fields=["ID_EXCH_SYMBOL"])
        blpts.get()
        blpts.closeSession()
        return str(blpts.output.ix[0, 0])
    except:
        return symbol

In [5]:
def ISIN(symbol):
    """
    Returns company name for a stoock
    
    symbol : bloomberg symbol, example 'AAPL US EQUITY'    
    """
    try:
        #sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[symbol],
                     fields=["ID_ISIN"])
        blpts.get()
        blpts.closeSession()
        return str(blpts.output.ix[0, 0])
    except:
        return symbol

In [6]:
def market_cap(symbol, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now()):
    """
    Returns pandas dataframe with market cap
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'
        
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    # symbol handler
    sym = symbol_handler(symbol)
    
    # download data
    period = 'DAILY'
    res = bw.simpleHistoryRequest(securities=[sym],\
                              fields = ["CUR_MKT_CAP"],\
                              startDate=start_date.date(),\
                              endDate=end_date.date(),\
                              periodicity=period)
    data = res[sym]
    data = data.apply(pd.to_numeric)
    data.index.name = 'Date'
    return data

In [61]:
def dividend_yield(symbol, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now()):
    """
    Returns pandas dataframe with dividend yeild
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'
        
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    # symbol handler
    sym = symbol_handler(symbol)
    
    # download data
    period = 'DAILY'
    res = bw.simpleHistoryRequest(securities=[sym],\
                              fields = ["EQY_DVD_YLD_12M"],\
                              startDate=start_date.date(),\
                              endDate=end_date.date(),\
                              periodicity=period)
    data = res[sym]
    data = data.apply(pd.to_numeric)
    data.index.name = 'Date'
    return data

In [7]:
def sector(symbol):
    """
    Returns sector stoock in
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'    
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    try:
        sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[sym],
                     fields=["INDUSTRY_SECTOR"])
        blpts.get()
        blpts.closeSession()
        return str(blpts.output.ix[0, 0])
    except:
        return symbol
    
def GICS_sector(symbol):
    """
    Returns sector stoock in
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'    
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    try:
        sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[sym],
                     fields=["GICS_SECTOR_NAME"])
        blpts.get()
        blpts.closeSession()
        return str(blpts.output.ix[0, 0])
    except:
        return symbol

In [8]:
"""
print company_name('US0138171014')
print ISIN('AA US EQUITY')
"""

"\nprint company_name('US0138171014')\nprint ISIN('AA US EQUITY')\n"

### Getting dividends history

In [9]:
def date_parser(d):
    """
    Returns datetime object from str with try-except handler
    
    d : date string in %Y-%m-%d format
    """
    try:
        return  datetime.datetime.strptime(d, '%Y-%m-%d')
    except:
        return np.nan

def get_dividends_and_splits_data(symbol):
    """
    Returns pandas dataframe with all data on historicaldividends
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'
        
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    try:
        # symbol handler
        sym = symbol_handler(symbol)
        blpts = bw.BLPTS(securities=[sym],
                     fields=["DVD_HIST_ALL"])
        blpts.get()
        blpts.closeSession()
        downloaded_data = blpts.output.ix[0, 0]
        dvd_array = np.array([list(map(lambda col: date_parser(dvd[col]),['Ex-Date', 'Declared Date', 'Record Date', 'Payable Date']))+\
                                  [dvd['Dividend Amount'], dvd['Dividend Frequency'], dvd['Dividend Type']] for dvd in downloaded_data])


        dvd_data = pd.DataFrame(dvd_array)
        dvd_data.columns = ['Ex-Date', 'Declared Date', 'Record Date', 'Payable Date','Dividend Amount', 'Dividend Frequency', \
                            'Dividend Type']
        
        dvd_data.set_index('Ex-Date', inplace=True)
        dvd_data['Dividend Amount'] = dvd_data['Dividend Amount'].apply(pd.to_numeric)
        #assert np.all(list(map(lambda x: x in ['Regular Cash', 'Special Cash', 'Stock Split', 'Pro Rata', 'Discontinued'], dvd_data['Dividend Type'])))

        return dvd_data
    except:
        return None

In [21]:
"""
# example
sym = 'IBM US EQUITY'
dvd_data = get_dividends_and_splits_data(sym)
dvd_data.head()
"""

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Using ISIN ID is recommended. You use IBM US EQUITY as a symbol


,Declared Date,Record Date,Payable Date,Dividend Amount,Dividend Frequency,Dividend Type
Ex-Date,,,,,,
2017-02-08,2017-01-31 00:00:00,2017-02-10 00:00:00,2017-03-10 00:00:00,1.4,Quarter,Regular Cash
2016-11-08,2016-10-25 00:00:00,2016-11-10 00:00:00,2016-12-10 00:00:00,1.4,Quarter,Regular Cash
2016-08-08,2016-07-26 00:00:00,2016-08-10 00:00:00,2016-09-10 00:00:00,1.4,Quarter,Regular Cash
2016-05-06,2016-04-26 00:00:00,2016-05-10 00:00:00,2016-06-10 00:00:00,1.4,Quarter,Regular Cash
2016-02-08,2016-01-26 00:00:00,2016-02-10 00:00:00,2016-03-10 00:00:00,1.3,Quarter,Regular Cash


In [11]:
"""
output_path = './market_data_spx/Errors/A'
universe = ['AA', 'RDC', 'SVU', 'WFT', 'CIT']
universe = list(map(lambda sym: sym+' US EQUITY', universe))
for sym in universe:
    dvd_data = get_dividends_and_splits_data(sym)
"""

"\noutput_path = './market_data_spx/Errors/A'\nuniverse = ['AA', 'RDC', 'SVU', 'WFT', 'CIT']\nuniverse = list(map(lambda sym: sym+' US EQUITY', universe))\nfor sym in universe:\n    dvd_data = get_dividends_and_splits_data(sym)\n"

### Base task: download price

In [12]:
def adjust_price_to_dividends(symbol, market_data):
    """
    Returns pandas dataframe with adjusted market data
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'
        
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    start_date = market_data.index[0]
    dvd_data = get_dividends_and_splits_data(symbol)
    if dvd_data is not None:
        # filter out splits
        dvd_data = dvd_data[dvd_data['Dividend Type'].isin(['Regular Cash', 'Special Cash'])]

        if dvd_data.shape[0] == 0 or pd.isnull(dvd_data.index).any():
            return market_data
        # TO DO: debgug next line
        dvd_data = dvd_data.loc[dvd_data.index <= market_data.index[-1]]
        # cumulative dividends column       
        dvd_data = dvd_data.groupby(by=dvd_data.index).agg({'Dividend Amount' : np.sum})
        dvd_data.sort_index(ascending=False, inplace=True)
        dvd_data['cum_dvds'] = dvd_data['Dividend Amount'].cumsum()      
        market_data = pd.concat([market_data, dvd_data['cum_dvds']], axis=1)
        market_data['cum_dvds'] = market_data['cum_dvds'].bfill()
        market_data['cum_dvds'] = market_data['cum_dvds'].shift(-1)
        market_data['cum_dvds'].fillna(0, inplace=True)

        market_data = market_data.ffill()
        # cut market data by time
        market_data = market_data.loc[market_data.index >= start_date]
        # subtract by cum-dividends
        for col in  ['Open', 'High', 'Low', 'Close']:
            market_data[col] = market_data[col] - market_data['cum_dvds']
        # we do not need contain data about cumulative dividends here
        market_data.drop('cum_dvds', axis=1, inplace=True)
    else: 
        warnings.warn("There is no dividends data on {}".format(symbol))  
    return market_data

In [13]:
def get_daily_price(symbol, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now(), \
              adjust_to_dividends=False):
    """
    Returns OHLC data for a given symbol
    
    symbol : ISIN ID or bloomberg symbol, example 'US0378331005'
    start_date : when data should start
    end_date : when data should end
    adjust_to_dividends : bool flag, by default function will return only splits adjusted price
    
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    # symbol handler
    sym = symbol_handler(symbol)
    
    # download data
    period = 'DAILY'
    res = bw.simpleHistoryRequest(securities=[sym],
                              fields = ["OPEN", "HIGH", "LOW", "LAST_PRICE"],
                              startDate=start_date.date(),
                              endDate=end_date.date(),
                              periodicity=period)
    market_data = res[sym]
    market_data = market_data[["OPEN", "HIGH", "LOW", "LAST_PRICE"]]
    market_data.index.name = 'Date'
    market_data.columns = ['Open', 'High', 'Low', 'Close']
    market_data = market_data.apply(pd.to_numeric)
    # adjust to dividends
    if adjust_to_dividends:
        market_data = adjust_price_to_dividends(symbol, market_data)
    market_data.index.name = 'Date'
    return market_data

In [14]:
"""
# example
sym = 'AAPL US EQUITY'
dvd_data = get_dividends_and_splits_data(sym)
dvd_data.head()
"""

"\n# example\nsym = 'AAPL US EQUITY'\ndvd_data = get_dividends_and_splits_data(sym)\ndvd_data.head()\n"

In [15]:
"""
# example
sym = 'AAPL US EQUITY'
market_data = get_daily_price(sym,adjust_to_dividends=True)
plt.figure(figsize=(10, 6))
plt.grid()
plt.title('{} close price'.format(sym))
plt.plot(market_data['Close'])
market_data.shape
"""

"\n# example\nsym = 'AAPL US EQUITY'\nmarket_data = get_daily_price(sym,adjust_to_dividends=True)\nplt.figure(figsize=(10, 6))\nplt.grid()\nplt.title('{} close price'.format(sym))\nplt.plot(market_data['Close'])\nmarket_data.shape\n"

In [16]:
"""
# example
sym = 'PKI US EQUITY'
market_data = get_daily_price(sym,adjust_to_dividends=False)
plt.figure(figsize=(10, 6))
plt.grid()
plt.title('{} close price'.format(sym))
plt.plot(market_data['Close'])
market_data.shape
"""

"\n# example\nsym = 'PKI US EQUITY'\nmarket_data = get_daily_price(sym,adjust_to_dividends=False)\nplt.figure(figsize=(10, 6))\nplt.grid()\nplt.title('{} close price'.format(sym))\nplt.plot(market_data['Close'])\nmarket_data.shape\n"

In [17]:
"""
# example
sym = 'PKI US EQUITY'
market_data = market_cap(sym)
plt.figure(figsize=(10, 6))
plt.grid()
plt.title('{} close price'.format(sym))
plt.plot(market_data)
market_data.shape
"""

"\n# example\nsym = 'PKI US EQUITY'\nmarket_data = market_cap(sym)\nplt.figure(figsize=(10, 6))\nplt.grid()\nplt.title('{} close price'.format(sym))\nplt.plot(market_data)\nmarket_data.shape\n"

### Common task: download data for universe

In [2]:
def bulk_market_data(universe, output_path, start_date=datetime.datetime(2006,1,1), end_date=datetime.datetime.now(), \
              adjust_to_dividends=False, update_flag=False):
    """
    Saves market data on a given symbol in folder provided
    
    universe : list with ISIN IDS or bloomberg symbols for tickers you need, example ['AAPL US EQUITY', 'FB US EQUITY']
    start_date : when data should start
    end_date : when data should end
    adjust_to_dividends : bool flag, by default function will return only splits adjusted price
        
    COMMENT:
    USING ISIN US RECOMMENDED
    """
    for sym in tqdm.tqdm(universe):
        sym_market_data = get_daily_price(sym, start_date=start_date, end_date=end_date, \
                                             adjust_to_dividends=adjust_to_dividends)
        if sym_market_data.shape[0] == 0:
            warnings.warn("Market data empty for {} symbol. Company NAME is {}".format(sym, company_name(sym)))   
        if not update_flag:
            sym_market_data.to_csv(output_path + '/{}.csv'.format(sym))
        else:
            read_args = {'parse_dates' : ['Date'], 'index_col' : 'Date'}
            prev_smdf = pd.read_csv(output_path + '/{}.csv'.format(sym), **read_args)
            
            sym_market_data = pd.concat([prev_smdf, sym_market_data], axis=0)
            
            sym_market_data = sym_market_data.groupby(sym_market_data.index).last()
            
            sym_market_data.to_csv(output_path + '/{}.csv'.format(sym))

NameError: name 'datetime' is not defined

In [19]:
"""
# example
bulk_market_data(
    ['AAPL US EQUITY', 'FB US EQUITY'],\
    'C:/Users/bloomberg/Documents/Python Scripts/Bloomberg download information/spx_market_data/'
)
"""

"\n# example\nbulk_market_data(\n    ['AAPL US EQUITY', 'FB US EQUITY'],    'C:/Users/bloomberg/Documents/Python Scripts/Bloomberg download information/spx_market_data/'\n)\n"